#**Install Groq**

In [3]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 9.7 MB/s eta 0:00:00


##**Loading Groq API key**

In [4]:
import os
from google.colab import userdata

# Access the API key from Colab secrets
groq_api_key = userdata.get('GROQ_API_KEY')

# You can set it as an environment variable, which some libraries might look for
# Or hard code if you are having issues
os.environ["GROQ_API_KEY"] = groq_api_key

print("Groq API key loaded successfully.")

Groq API key loaded successfully.


##**Task 2: Appointment Booking Bot (Student Take-Home Work)**
## Make the appointment booking bot conversational by adding state or a concept of history.

##Modifications:

###Modify the process_appointment_request function or the surrounding loop to maintain a history of the conversation.
###Use the conversation history in subsequent prompts to the LLM so that it can understand the context of the current user request.
###The bot should be able to:
###Remember previous turns in the conversation (e.g., if the user mentioned a preferred time in a previous turn).
###Ask clarifying questions based on the conversation history.
###Ultimately, confirm the appointment details by referring back to the information gathered throughout the conversation.
###Consider implementing a simple state machine or using a list to store message history.

In [5]:
import os
from groq import Groq

# ===============================
# Load Groq API Key Securely
# ===============================

# 1️⃣ Try Colab secrets (if running in Colab)
try:
    from google.colab import userdata
    groq_api_key = userdata.get("GROQ_API_KEY")
except (ImportError, Exception):
    # 2️⃣ Try Kaggle or system environment variable
    groq_api_key = os.getenv("GROQ_API_KEY")

# 3️⃣ Optional: Load from a local .env file (if running locally)
if not groq_api_key:
    try:
        from dotenv import load_dotenv
        load_dotenv()
        groq_api_key = os.getenv("GROQ_API_KEY")
    except ImportError:
        pass

# 4️⃣ Final safety check
if not groq_api_key or groq_api_key.strip() == "":
    raise ValueError("❌ GROQ_API_KEY not found! Please set it in Colab/Kaggle secrets or .env file.")

# Set it as an environment variable for Groq library
os.environ["GROQ_API_KEY"] = groq_api_key

# ===============================
# Initialize Groq Client
# ===============================
client = Groq(api_key=groq_api_key)

# Conversation history
conversation_history = [
    {
        "role": "system",
        "content": (
            "You are an appointment booking assistant. Have a conversation with the user to gather "
            "all necessary details (like date, time, type of appointment). Confirm the appointment "
            "once all information is complete. Ask for clarification if needed. Be polite and helpful."
        ),
    }
]

def process_appointment_request(user_input):
    """Processes user input for an appointment using Llama 70B with history."""
    try:
        # Add user input to history
        conversation_history.append({"role": "user", "content": user_input})

        # Call the model with conversation history
        chat_completion = client.chat.completions.create(
            messages=conversation_history,
            model="llama3-70b-8192",
        )

        # Get assistant response
        response = chat_completion.choices[0].message.content

        # Append assistant response to history
        conversation_history.append({"role": "assistant", "content": response})

        return response
    except Exception as e:
        return f"An error occurred: {e}"

# ===============================
# Chat loop
# ===============================
print("Hello! I am your appointment booking bot. Let's get started.")
print("Type 'quit' to exit.\n")

while True:
    user_request = input("You: ").strip()
    if user_request.lower() == 'quit':
        print("Bot: Goodbye! Have a great day.")
        break

    bot_response = process_appointment_request(user_request)
    print("Bot:", bot_response)


Hello! I am your appointment booking bot. Let's get started.
Type 'quit' to exit.

You: quit
Bot: Goodbye! Have a great day.
